<a href="https://colab.research.google.com/github/rahulhalderAERO/Hydrodynamic_LSTM_ROM/blob/main/Hydrodynamic_PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The LSTM-PINN algorithms introduces the loss function computed from the discretized governing equation.The Governing Equation is the rigid body dynamic Equation as $M\ddot{x} + K {x} = F$


In [1]:
from keras.models import Sequential 
from keras.layers import Dense
import numpy as np
from matplotlib import pyplot as plt
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from keras import backend as K
import tensorflow as tf
import random
import time
start_time = time.time()

Import Training and Test Data

In [3]:
Output = pd.read_csv("box_smooth.csv",skiprows = None , header = None )
Input = pd.read_csv("Test_Input_Force_Only.csv",skiprows = None , header = None )
Output1 = pd.read_csv("box_smooth.csv",skiprows = None , header = None )
Input1 = pd.read_csv("Test_Input_Force_Only.csv",skiprows = None , header = None )